Get all Swedish Bathing Waters in this project from Wikidata and checks if they are connected to an OSM object using an [API](https://osm.wikidata.link/tagged)

* [github salgo60/Svenskabadplatser](https://github.com/salgo60/Svenskabadplatser)
* this [Notebook](https://github.com/salgo60/Svenskabadplatser/blob/main/Jupyter/OSM_Wikidata_Bathwater.ipynb)

* API [Wikidata to OpenStreetMap](https://osm.wikidata.link/tagged)
  * ex [https://osm.wikidata.link/tagged/api/item/Q106708773](https://osm.wikidata.link/tagged/api/item/Q106708773)
* Wikidata badplats <-> Naturreservat [SPARQL](https://w.wiki/3MwX)
  * Lesson learned: inte alla Naturreservat finns på Open Street Map

* Another tool [osm.wikidata.link](https://osm.wikidata.link/search)
  
Status:  



| Date | Badplatser | ej WD - OSM | Vatten | ej WD - OSM |naturreservat | ej WD - OSM  |
| ------------- |:-------------:|:-------------:|:-------------:|:-------------:|:-------------:| -----:|
| 20210529 | 2812 | 1946 | 1149 | 197 | 214 | 16
| 20210528 | 2811 | 1953 | 1149 | 196 | 214 | 16
| 20210527 | 2809 | 1991 | 1228 | 232 | 214 | 16
| 20210526 | 2806 | 2029 | 1202 | 235 | 213 | 16
| 20210525     | 2789 | 2076 |1104 | 234 | 209|84|
| 20210524     | 2787 | 2787 |1094 | 230 | 208|84| 
| 20210523     | 2766 | 2120 |1030 | 219 | 205|86|
| 20210522     | 2761 | 2142 |1001 | 216 | 205|87|
| 20210521     | 2756 | 2199 |979 | 215 | ||
| 20210519     | 2757 | 2257 |594 | 59 | ||
| 20210518     | 2755 | 2327 |484 | 60 | ||

TODO: 
* Vatten är sjöar och Kustvatten. Kustvatten verkar inte finnas i OSM så kanske skilja dom åt

In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2021-05-29 14:48:59.665073


In [2]:
import pandas as pd


In [3]:
#
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys,json
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"
 
# https://w.wiki/3LWk    
queryBath = """SELECT (REPLACE(STR(?nodeBath), ".*Q", "Q") AS ?qbathid) ?nodeBathLabel ?nutsCode ?SJOID
(REPLACE(STR(?nodeWater), ".*Q", "Q") AS ?qlakeid) ?nodeWaterLabel ?coord WHERE {
  ?nodeBath wdt:P6104 wd:Q106774536.
  OPTIONAL { ?nodeBath wdt:P605 ?nutsCode. }
  OPTIONAL { ?nodeBath wdt:P625 ?coord. }
  OPTIONAL {
    ?nodeBath wdt:P206 ?nodeWater.
    OPTIONAL { ?nodeWater wdt:P761 ?SJOID. }
  }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
}
ORDER BY (?nodeBathLabel)"""


def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

WDBath = get_sparql_dataframe(endpoint_url, queryBath)
WDBath["Source"] = "WD"     
WDBath.shape

(2812, 8)

In [4]:
WDBath.head()

,qbathid,nodeBathLabel,nutsCode,SJOID,qlakeid,nodeWaterLabel,coord,Source
0,Q106708773,Abborrbergets badplats,SE0220486000001903,658887-156656,Q35694946,Strängnäsfjärden,Point(17.037397412 59.384608492),WD
1,Q106707080,Abborrsjöns badplats,SE0411060000000277,624471-143063,Q16275305,Abborrasjön,Point(14.691359 56.326564),WD
2,Q106707080,Abborrsjöns badplats,SE0411060000000277,624471-143063,Q16275305,Abborrasjön,Point(14.691358723 56.326564061),WD
3,Q106711299,Abborrtjärns badplats,SE0812404000003683,713089-168959,Q16487572,Abborrtjärnen,Point(19.724889698 64.231408958),WD
4,Q106708954,Abborrtjärns badplats,SE0611763000002279,660121-136542,Q16275388,Abborrtjärn,Point(13.4333 59.5096),WD


In [5]:
import urllib3, json
from tqdm import tqdm
http = urllib3.PoolManager()

listBath = []
#for WD, row in WDBath.iterrows():
for WD, row in tqdm(WDBath.iterrows(), total=WDBath.shape[0]):
    #print(row["qbathid"] ) 
    url = "https://osm.wikidata.link/tagged/api/item/" + row["qbathid"] 
    
    new_item = dict()
    new_item['wikidata'] = row["qbathid"] 
    new_item['coord'] = row["coord"] 
    r = http.request('GET', url) 
    data = json.loads(r.data.decode('utf-8'))
#    print (r.status)
    try:
        #print ("OSM ", data["osm"], "Type: ", type(data["osm"]))
        #print ("ID: ", data["osm"][0]["id"])
        osmid = data["osm"][0]["id"]        
    except:
        #print ("error")
        osmid =""
    new_item['osmid'] = osmid 
    listBath.append(new_item)
print (len(listBath))

100%|██████████| 2812/2812 [04:17<00:00, 10.91it/s]

2812


In [6]:
OSMtot = pd.DataFrame(listBath,
                  columns=['wikidata','coord','osmid'])
OSMtot.shape


(2812, 3)

In [7]:
pd.set_option('max_colwidth', 400)
OSMtot.head(10)

,wikidata,coord,osmid
0,Q106708773,Point(17.037397412 59.384608492),8725492343
1,Q106707080,Point(14.691359 56.326564),8737463766
2,Q106707080,Point(14.691358723 56.326564061),8737463766
3,Q106711299,Point(19.724889698 64.231408958),8763990476
4,Q106708954,Point(13.4333 59.5096),356838031
5,Q106710481,Point(18.576809583 65.354219925),943675486
6,Q106708549,Point(14.397042778 57.006691546),8763997383
7,Q106708324,Point(14.155285315 56.744170005),4428909177
8,Q106708011,Point(13.779767114 57.251420376),8737439015
9,Q106689287,Point(18.4089 59.2016),5347857709


In [8]:
#OSMempty = OSMtot.osmid.notnull()
OSMtot[(OSMtot['osmid']=="")].shape

(1958, 3)

In [9]:
OSMEmpty =OSMtot[(OSMtot['osmid']=="")]

In [10]:
OSMEmpty.shape

(1958, 3)

In [11]:
OSMEmpty.to_csv("WD - OSM Badplatser saknas.csv")

OSMEmpty.head()

,wikidata,coord,osmid
12,Q106709184,Point(14.414252823 59.404268046),
16,Q106708747,Point(18.158786988 59.25280499),
24,Q106709070,Point(14.869880056 60.749363203),
25,Q106708712,Point(15.445449575 58.121240586),
27,Q106708053,Point(14.812960627 57.823274799),


### Check sjöar

In [12]:
querySeaMinusKust = """SELECT (REPLACE(STR(?nodeBath), ".*Q", "Q") AS ?qbathid) ?nodeBathLabel ?nutsCode ?SJOID
(REPLACE(STR(?nodeWater), ".*Q", "Q") AS ?qlakeid) ?nodeWaterLabel ?coord WHERE {
  ?nodeBath wdt:P6104 wd:Q106774536.
  OPTIONAL { ?nodeBath wdt:P605 ?nutsCode. }
  OPTIONAL { ?nodeBath wdt:P625 ?coord. }
   {
    ?nodeBath wdt:P206 ?nodeWater.
    OPTIONAL { ?nodeWater wdt:P761 ?SJOID. }
    minus {?nodeWater wdt:P31 wd:Q16615865}
  }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
}
ORDER BY (?nodeBathLabel)"""
# https://w.wiki/3LqW
querySea = """SELECT (REPLACE(STR(?nodeBath), ".*Q", "Q") AS ?qbathid) ?nodeBathLabel ?nutsCode ?SJOID
(REPLACE(STR(?nodeWater), ".*Q", "Q") AS ?qlakeid) ?nodeWaterLabel ?coord WHERE {
  ?nodeBath wdt:P6104 wd:Q106774536.
  OPTIONAL { ?nodeBath wdt:P605 ?nutsCode. }
  OPTIONAL { ?nodeBath wdt:P625 ?coord. }
   {
    ?nodeBath wdt:P206 ?nodeWater.
    OPTIONAL { ?nodeWater wdt:P761 ?SJOID. }
  }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
}
ORDER BY (?nodeBathLabel)"""
WDSea = get_sparql_dataframe(endpoint_url, querySeaMinusKust)
WDSea["Source"] = "WD"     
WDSea.shape

(1149, 8)

In [13]:
WDSea.head()

,qbathid,nodeBathLabel,nutsCode,SJOID,qlakeid,nodeWaterLabel,coord,Source
0,Q106708773,Abborrbergets badplats,SE0220486000001903,658887-156656,Q35694946,Strängnäsfjärden,Point(17.037397412 59.384608492),WD
1,Q106707080,Abborrsjöns badplats,SE0411060000000277,624471-143063,Q16275305,Abborrasjön,Point(14.691359 56.326564),WD
2,Q106707080,Abborrsjöns badplats,SE0411060000000277,624471-143063,Q16275305,Abborrasjön,Point(14.691358723 56.326564061),WD
3,Q106708954,Abborrtjärns badplats,SE0611763000002279,660121-136542,Q16275388,Abborrtjärn,Point(13.4333 59.5096),WD
4,Q106711299,Abborrtjärns badplats,SE0812404000003683,713089-168959,Q16487572,Abborrtjärnen,Point(19.724889698 64.231408958),WD


In [14]:
listSea = []
#for WD, row in WDBath.iterrows():
for WD, row in tqdm(WDSea.iterrows(), total=WDBath.shape[0]):
    #print(row["qlakeid"] ) 
    url = "https://osm.wikidata.link/tagged/api/item/" + row["qlakeid"] 
    
    new_item = dict()
    new_item['wikidata'] = row["qlakeid"] 
    r = http.request('GET', url) 
    data = json.loads(r.data.decode('utf-8'))
    try:
        #print ("ID: ", data["osm"][0]["id"])

        osmid = data["osm"][0]["id"] 
        
    except:
        #print ("error")
        osmid =""
    new_item['osmid'] = osmid 
    listSea.append(new_item)
print (len(listSea))
OSMSeatot = pd.DataFrame(listSea,
                  columns=['wikidata','osmid'])

 41%|████      | 1149/2812 [01:46<02:34, 10.75it/s]

1149


In [15]:
OSMSeatot.head()

,wikidata,osmid
0,Q35694946,
1,Q16275305,10677610
2,Q16275305,10677610
3,Q16275388,241597601
4,Q16487572,30167501


In [16]:
OSMSeaEmpty=OSMSeatot[(OSMSeatot['osmid']=="")]
# OSMSeaEmpty.wikidata.unique()  
#pd.Series(OSMSeaEmpty.wikidata.unique().sort(),dtype=pd.StringDtype()).to_csv("WD - OSM Vatten saknas.csv")
pd.Series(OSMSeaEmpty.wikidata.unique()).to_csv("WD - OSM Vatten saknas.csv")

OSMSeaEmpty.shape


(234, 2)

# Check Naturreservat

In [17]:
# https://w.wiki/3N6L
queryNature = """SELECT (REPLACE(STR(?nodeBath), ".*Q", "Q") AS ?qbathid) ?nodeBathLabel ?nutsCode ?Naturreg
(REPLACE(STR(?naturreservat), ".*Q", "Q") AS ?naturreservatID)  WHERE {
  ?nodeBath wdt:P6104 wd:Q106774536.
  OPTIONAL {?nodeBath wdt:P605 ?nutsCode }
  ?nodeBath wdt:P276 ?naturreservat.
  ?naturreservat wdt:P3613 ?Naturreg. # --> reservat
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
}
"""
WDNature = get_sparql_dataframe(endpoint_url, queryNature)
WDNature["Source"] = "WD"     
WDNature.shape

(214, 6)

In [18]:
WDNature.head()

,qbathid,nodeBathLabel,nutsCode,Naturreg,naturreservatID,Source
0,Q106707117,badplats Larödbaden,SE0441283000000317,2045268,Q60732601,WD
1,Q106707119,Domsten,SE0441283000000319,2030989,Q29579930,WD
2,Q106707120,Utvälinges badplats,SE0441283000000320,2042687,Q10659042,WD
3,Q106707127,Gyllebosjöns badplats,SE0441291000000329,2014529,Q10511301,WD
4,Q106707141,badplats Ryssvägen,SE0441286000000348,2001805,Q27968545,WD


In [19]:
listNature = []
for WD, row in tqdm(WDNature.iterrows(), total=WDNature.shape[0]):
    #print(row["naturreservatID"] ) 
    url = "https://osm.wikidata.link/tagged/api/item/" + row["naturreservatID"] 
    
    new_item = dict()
    new_item['wikidata'] = row["naturreservatID"] 
    r = http.request('GET', url) 
    data = json.loads(r.data.decode('utf-8'))
    try:
        #print ("ID: ", data["osm"][0]["id"])
        osmid = data["osm"][0]["id"] 
        
    except:
        #print ("error")
        osmid =""
    new_item['osmid'] = osmid 
    listNature.append(new_item)
print (len(listNature))
OSMNature = pd.DataFrame(listNature,
                  columns=['wikidata','osmid'])

100%|██████████| 214/214 [00:20<00:00, 10.52it/s]

214


In [20]:
OSMNature.head()

,wikidata,osmid
0,Q60732601,
1,Q29579930,913660002
2,Q10659042,12400357
3,Q10511301,1460544
4,Q27968545,102924516


In [21]:
OSMNatureEmpty=OSMNature[(OSMNature['osmid']=="")]
#OSMNatureEmpty.to_csv("WD - OSM Naturereservat saknas.csv")
pd.Series(OSMNatureEmpty.wikidata.unique()).to_csv("WD - OSM Naturereservat saknas.csv")

#OSMNatureEmpty

In [22]:
print("*", start_time.strftime("%Y%m%d"),"Badplatser", WDBath.shape[0], "ej OSM kopplade",OSMEmpty.shape[0]) 
print(" * vatten kopplade", WDSea.shape[0], "ej OSM kopplade",OSMSeaEmpty.shape[0])
print(" * naturreservat kopplade", OSMNature.shape[0], "ej OSM kopplade",OSMNatureEmpty.shape[0])


* 20210529 Badplatser 2812 ej OSM kopplade 1958
 * vatten kopplade 1149 ej OSM kopplade 234
 * naturreservat kopplade 214 ej OSM kopplade 28


Line for GITHUB summary

In [23]:
print("ej kopplade",OSMEmpty.shape[0],"vatten ej kopplade", OSMSeaEmpty.shape[0], \
      ", naturreservat ej kopplade",OSMNatureEmpty.shape[0])



ej kopplade 1958 vatten ej kopplade 234 , naturreservat ej kopplade 28


Generate Markdown table eg.
| 20210526     | 2802 | 2050 |1147 | 254 | 213| 84|


In [24]:
print("|",start_time.strftime("%Y%m%d"),"|", \
      WDBath.shape[0],"|",OSMEmpty.wikidata.nunique(),"|", \
      OSMSeatot.shape[0],"|", OSMSeaEmpty.wikidata.nunique(),"|", \
      OSMNature.shape[0],"|", OSMNatureEmpty.wikidata.nunique())


| 20210529 | 2812 | 1946 | 1149 | 197 | 214 | 16
